In [ ]:
#mount google drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')
main_folder = ""

In [ ]:
import os
import copy
import itertools
import logging
import glob

from functools import partial
from datetime import datetime
from random import randint,shuffle

import tensorflow as tf
import numpy as np
from numpy import genfromtxt
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

import matplotlib.pyplot as plt

## Data Utils

In [ ]:

def denoise(coords, input_dim):
    stroke = savgol_filter(coords[:, 0], 7, 3, mode='nearest')
    for i in range(1, input_dim):
        x_new = savgol_filter(coords[:, i], 7, 3, mode='nearest')
        stroke = np.hstack([stroke.reshape(len(coords), -1), x_new.reshape(-1, 1)])
    return stroke

def denoise_all(stroke_all, input_dim):
    """
    smoothing filter to mitigate some artifacts of the data collection
    """
    stroke_new_all = []
    for coords in stroke_all:
        stroke = denoise(coords, input_dim)
        stroke_new_all.append(stroke)
    return stroke_new_all

def interpolate_all(stroke_all, max_x_length, input_dim):
    """
    interpolates strokes using cubic spline
    """
    coords_all = []
    for stroke in stroke_all:
        coords = interpolate(stroke, max_x_length, input_dim)
        coords_all.append(coords)
    return coords_all

def interpolate(stroke, max_x_length, input_dim):
    coords = np.zeros([input_dim, max_x_length], dtype=np.float32)
    if len(stroke) > 3:
        for j in range(input_dim):
            f_x = interp1d(np.arange(len(stroke)), stroke[:, j], kind='linear')
            xx = np.linspace(0, len(stroke) - 1, max_x_length)
            # xx = np.random.uniform(0,len(stroke)-1, max_x_length)
            x_new = f_x(xx)
            coords[j, :] = x_new
    coords = np.transpose(coords)
    return coords

def multiplier(data,label,multi):
    data_ = data
    label_ = label
    for i in range(multi):
        data_ = np.concatenate((data_, data))
        label_ = np.concatenate((label_, label))

    data = data_
    label = label_

    return data,label


def shuffle(data,label):
    data = np.asarray(data,dtype = object)
    shuffled_indexes = np.random.permutation(np.shape(data)[0])
    data = data[shuffled_indexes]
    label = label[shuffled_indexes]
    return data, label


def get_new_gesture_data(path,max_length,input_dim, data_synthesis = False):
    new_gesture_list = []

    for i, fname in enumerate(sorted(glob.glob(path))):
        new_gesture = genfromtxt(fname, delimiter=',')
        #     new_gesture = denoise(new_gesture,input_dim)
        new_gesture = interpolate(new_gesture, max_length, input_dim)
        new_gesture_list.append(new_gesture)

    if data_synthesis:
        synthetic_data = np.load('synthetic_data/synthetic_data.npy', allow_pickle=True)
        new_gesture_list = np.concatenate((new_gesture_list, synthetic_data))
    new_label_list = np.ones(len(new_gesture_list)) * 11
    new_gesture_list , new_label_list = shuffle(new_gesture_list,new_label_list)
    return new_gesture_list , new_label_list

def train_test_split(data,label,new_gesture_list,new_label_list,split_num):

    for _ in range(10):
        data, label =  shuffle(data, label)

    train_data, test_data = np.split(
        data, [np.shape(data)[0] * 9 // 10]
    )
    train_label, test_label = np.split(
        label, [np.shape(label)[0] * 9 // 10]
    )
    multi = 10

    train_data = np.concatenate((train_data,new_gesture_list[:split_num]))
    train_label = np.concatenate((train_label,new_label_list[:split_num]))
    train_data, train_label = multiplier(train_data, train_label, multi)

    test_data = np.concatenate((test_data,new_gesture_list[split_num:]))
    test_label = np.concatenate((test_label,new_label_list[split_num:]))
    test_data, test_label = multiplier(test_data, test_label, multi)

    train_data, train_label = shuffle(train_data, train_label)

    test_data, test_label = shuffle(test_data, test_label)

    train_data = np.asarray(train_data, dtype=np.float32)
    test_data = np.asarray(test_data, dtype=np.float32)

    train_label = np.asarray(train_label, dtype=np.int32)
    test_label = np.asarray(test_label, dtype=np.int32)

    return train_data, train_label, test_data, test_label

def confusion_matrix(save_path,new_gesture):
    cm = np.load(save_path + '/confusion_matrix.npy')

    classes = ['still', 'ticktock', 'shrink', 'push', 'peaceout', 'madriddles',
               'grow', 'flamingo', 'execution', 'cheshiredance', 'caterpillar', new_gesture]

    target_names = list(classes)
    title = 'Confusion matrix'
    cmap = None
    normalize = True

    accuracy = np.trace(cm) / float(np.sum(cm))
    # np.save(SAVE_PATH+ 'confusion_matrix.npy',cm)
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(10, 10))
    # plt.imshow(cm, interpolation='nearest', cmap=cmap)
    # plt.title(title, fontsize=30)
    # plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90, fontsize=20)
        plt.yticks(tick_marks, target_names, fontsize=20)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 10 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.imshow(cm, cmap=cmap)
    # plt.grid(None)
    plt.tight_layout()
    # plt.ylabel('True label', fontsize=50)
    # plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass), fontsize=30)
    # plt.show()
    plt.savefig('confusion_matrix.pdf')
    # plt.close()


def model_selection(Model,drop_out,max_length,input_dim ,output_dim):
    model = ''

    if Model == 'rnn_att_model':
        model = rnn_att_model(max_length,
                          input_dim,
                          output_dim,dropout=drop_out)

    if Model == 'resNet_LSTM':
        model = resNet_LSTM(max_length,
                          input_dim,
                          output_dim,dropout=drop_out)

    elif Model == 'resNet':
        model = resNet(max_length,
                          input_dim,
                          output_dim)
    elif Model == 'cnnModel':
        model = cnnModel(max_length,
                          input_dim,
                          output_dim)
    elif Model == 'Inception':
        model = Inception(max_length,
                          input_dim,
                          output_dim)
    elif Model == 'smallCnnModel':
        model = smallCnnModel(max_length,
                          input_dim,
                          output_dim)

    elif Model == 'LSTM_RES':
        model = LSTM_RES(max_length,
                          input_dim,
                          output_dim,dropout=drop_out)

    return  model


def init_logging(log_dir, Model):
    logging_level = logging.INFO

    log_file = 'log_{}.txt'.format(Model)

    log_file = os.path.join(log_dir, log_file)
    if os.path.isfile(log_file):
        os.remove(log_file)

    logging.basicConfig(
        filename=log_file,
        level=logging_level,
        format='[[%(asctime)s]] %(message)s',
        datefmt='%m/%d/%Y %I:%M:%S %p'
    )
    logging.getLogger().addHandler(logging.StreamHandler())

    return logging


## Data Handling

In [ ]:
def read_shrec_2021(path):
    gesture_data = []
    sequence_idx_list = []

    for i, fname in enumerate(sorted(glob.glob(path))):
        # print(fname)

        idx = fname.split('/')[-1].split('.')[0]
        # print(idx)

        data = genfromtxt(fname, delimiter=';')
        data_ = np.delete(data, -1, axis=1)
        gesture_data.append(data_)
        sequence_idx_list.append(int(idx))

    gesture_data_sorted = [x for _, x in sorted(zip(sequence_idx_list, gesture_data))]

    return gesture_data_sorted

def read_annotation(fname_,gesture_data,max_length = 80):
    gesture_name = ['ONE','TWO','THREE','FOUR','OK','MENU', 'LEFT',
                    'RIGHT','CIRCLE','V','CROSS','GRAB','PINCH','TAP','DENY', 'KNOB','EXPAND']
    with open(fname_) as f:
        lines = f.readlines()

    gesture_name_list = []
    start_idx_list = []
    end_idx_list = []
    annotation_idx_list = []
    for i in range(len(lines)):

        gesture_name_list.append(lines[i].split(';')[1:-1][::3])
        start_idx_list.append(lines[i].split(';')[2:-1][::3])
        end_idx_list.append(lines[i].split(';')[0:-1][::3][1:])
        annotation_idx_list.append(int(lines[i].split(';')[0]))

    gesture_name_list = [x for _, x in sorted(zip(annotation_idx_list, gesture_name_list))]
    start_idx_list = [x for _, x in sorted(zip(annotation_idx_list, start_idx_list))]
    end_idx_list = [x for _, x in sorted(zip(annotation_idx_list, end_idx_list))]


    single_gesture = []
    gesture_class = []

    for i in range(len(gesture_data)):
        for j in range(len(gesture_name_list[i])):
            start_idx = int(start_idx_list[i][j])
            end_idx = int(end_idx_list[i][j])
            single_gesture.append(gesture_data[i][start_idx:start_idx+max_length])
            gesture_class.append(gesture_name.index(gesture_name_list[i][j]))

    return single_gesture, gesture_class

def pad_data(input_dim, data,max_length):
    data_padded = np.zeros([len(data),max_length,input_dim])
    for i in range(len(data)):
        if len(data[i]) <= max_length:
            data_padded[i,:len(data[i])] = data[i]
        if len(data[i]) > max_length:
            data_padded[i] = data[i][:max_length]
    return data_padded

def select_joint(selected_joint,gesture_data,pos = True, quat= True):
    split_joint_name = ['palm', 'thumbA', 'thumbB', 'thumbEnd', 'indexA', 'indexB', 'indexC', 'indexEnd', 'middleA', 'middleB', 'middleC', 'middleEnd', 'ringA', 'ringB', 'ringC', 'ringEnd', 'pinkyA', 'pinkyB', 'pinkyC', 'pinkyEnd']

    selected_joint_idx = []
    for x in selected_joint:
        selected_joint_idx.append(split_joint_name.index(x))

    idx_list_pos_x = np.multiply(selected_joint_idx,7)
    idx_list_pos_y = np.multiply(selected_joint_idx,7)+1
    idx_list_pos_z = np.multiply(selected_joint_idx,7)+2
    idx_list_quat_x = np.multiply(selected_joint_idx,7)+3
    idx_list_quat_y = np.multiply(selected_joint_idx,7)+4
    idx_list_quat_z = np.multiply(selected_joint_idx,7)+5
    idx_list_quat_w = np.multiply(selected_joint_idx,7)+6

    idx_list_pos = np.concatenate([idx_list_pos_x,idx_list_pos_y,idx_list_pos_z])
    idx_list_quat = np.concatenate([idx_list_quat_x,idx_list_quat_y,idx_list_quat_z,idx_list_quat_w])


    if pos:
        data = gesture_data[:,sorted(idx_list_pos)]
    if quat:
        data = gesture_data[:,sorted(idx_list_quat)]
    if pos and quat:
        data = gesture_data[:,sorted(np.concatenate([idx_list_pos,idx_list_quat]))]

    return data

def preprocess(data, xlist = [0, 3, 6, 9, 12, 15],nor_to_wrist=False, relative=False):
    # relative replacement or not
    # use center as track or not, if not, use wrist as track
    length = np.shape(data)[0]
    input_dim = np.shape(data)[1]

    if relative:
        if nor_to_wrist:
            data = nortowrist(data, xlist)
            data = relative_track_batch(length, input_dim, data)
        else:
            data = relative_track_batch(length, input_dim, data)
    elif not relative:
        if nor_to_wrist:
            data = nortowrist(data, xlist)
        else:
            data = data

    return data

def relative_track_batch(length, input_dim, data):
    ### do this every iteration to make first step replacement=0
    temp = data[:, 0:input_dim]
    lastplace = np.zeros([length, 1])
    # x
    lastplace[1: length, 0] = temp[0: length - 1, 0]
    lastplace[0, 0] = temp[0, 0]
    temp[0: length, 0] -= lastplace[:, 0]
    # y
    lastplace[1: length, 0] = temp[0: length - 1, 1]
    lastplace[0, 0] = temp[0, 1]
    temp[0: length, 1] -= lastplace[:, 0]
    # z
    lastplace[1: length, 0] = temp[0: length - 1, 2]
    lastplace[0, 0] = temp[0, 2]
    temp[0: length, 2] -= lastplace[:, 0]
    temp = np.reshape(temp, [length, input_dim])
    return temp


def nortowrist(data, xlist):
    ###### normalize to wrist
    ylist = np.add(xlist, 1)
    zlist = np.add(xlist, 2)

    xc = data[:, 0]
    yc = data[:, 1]
    zc = data[:, 2]
    data[:, xlist] -= np.tile(
        np.reshape(xc, [-1, 1]), (1, len(xlist)))
    data[:, ylist] -= np.tile(
        np.reshape(yc, [-1, 1]), (1, len(ylist)))
    data[:, zlist] -= np.tile(
        np.reshape(zc, [-1, 1]), (1, len(zlist)))

    return data



In [ ]:
def tfdata(train_data, train_label, test_data, test_label, batch_size):


    trainData =  tf.data.Dataset.from_tensor_slices(
        (train_data, train_label)
    )
    trainData = trainData.shuffle(buffer_size=np.shape(train_data)[0])
    trainData = trainData.batch(batch_size).prefetch(buffer_size=1)
    trainsteps = np.shape(train_data)[0] // batch_size

    testData =  tf.data.Dataset.from_tensor_slices(
        (test_data, test_label)
    )
    testData = testData.shuffle(buffer_size=np.shape(test_data)[0])
    testData = testData.batch(batch_size).prefetch(buffer_size=1)
    teststeps = np.shape(test_data)[0] // batch_size

    return trainData,trainsteps, testData, teststeps


def getDataTest(test_data, test_label, batch_size):

    testData =  tf.data.Dataset.from_tensor_slices(
        (test_data, test_label)
    )

    testData = testData.batch(batch_size).prefetch(buffer_size=1)
    teststeps = np.shape(test_data)[0] // batch_size

    return testData, teststeps, test_data, test_label



## Training Function Implementation

In [ ]:
import pickle
def train_classifier(model, Model, SAVE_PATH, EPOCHS,
                learning_rate, train_data, train_label, test_data, test_label, batch_size):
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=["sparse_categorical_accuracy"])
    model.summary()

    SAVE_PATH_model = SAVE_PATH + 'Model/'

    trainData, trainsteps, \
    validData, validsteps,\
        = tfdata(train_data, train_label, test_data, test_label, batch_size)


    # Stop if the validation accuracy doesn't imporove for x epochs
    earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, verbose=2)

    # Reduce LR on Plateau

    reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=2)

    history_lstm = model.fit(trainData.repeat(),
                             steps_per_epoch=trainsteps,
                             validation_data=validData.repeat(),
                             validation_steps=validsteps,
                             epochs=EPOCHS,
                             # callbacks=[reduceLR,earlyStopping]
                             #callbacks=[reduceLR]
                             )

    ## Save model
    model.save(SAVE_PATH_model)

    ## Save model
    with open(SAVE_PATH_model + "train_results.pickle", "wb") as handle:
        pickle.dump(history_lstm.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Saved training history to res")


## Model Implementations

In [ ]:
'''
Models implementations
'''
def cnnModel(input_shape1,
                input_shape2,
                  output_shape):

    input_shape = (input_shape1, input_shape2)
    model = tf.keras.models.Sequential()

    # Normalization layer
    model.add(tf.keras.layers.Reshape(input_shape=input_shape, target_shape=(input_shape1, input_shape2, 1)))
    model.add(tf.keras.layers.BatchNormalization())

    filters = [16, 32, 64, 128]

    for num_filters in filters:
        # Conv a
        model.add(tf.keras.layers.Conv2D(
            num_filters,
            kernel_size=(3, 3),
            padding='same'
            )
        )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation('relu'))

        # Conv b
        model.add(tf.keras.layers.Conv2D(
            num_filters,
            kernel_size=(3, 3),
            padding='same'
            )
        )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation('relu'))

        # Pooling
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
        model.add(tf.keras.layers.Dropout(0.2))

    # Classification layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, name='features512'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(256, name='features256'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(output_shape, activation='softmax'))

    return model


def smallCnnModel(input_shape1,
                input_shape2,
                  output_shape):


    input_shape = (input_shape1, input_shape2)

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Reshape(input_shape=input_shape, target_shape=(input_shape1, input_shape2, 1)))

    model.add(tf.keras.layers.Convolution2D(32, (1, 10), padding='same', activation='relu'))
    model.add(tf.keras.layers.Convolution2D(64, (1, 5), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((1, 4)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Convolution2D(64, (1, 10), padding='valid', activation='relu'))
    model.add(tf.keras.layers.Convolution2D(128, (10, 1), padding='same', activation='relu'))

    model.add(tf.keras.layers.GlobalMaxPool2D())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(128, activation='relu', name='FEATURES'))
    model.add(tf.keras.layers.Dense(output_shape, activation='softmax'))

    return model


def modular_cnn_model(input_shape=(300, 18)):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Reshape(
        input_shape=input_shape, target_shape=(input_shape[0], input_shape[1], 1)))

    model.add(tf.keras.layers.Convolution2D(
        32, (1, 5), padding='same', activation='relu'))
    model.add(tf.keras.layers.Convolution2D(
        64, (1, 5), padding='same', activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((1, 4)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Convolution2D(
        64, (1, 5), padding='valid', activation='relu'))
    model.add(tf.keras.layers.Convolution2D(
        128, (5, 1), padding='same', activation='relu'))

    model.add(tf.keras.layers.GlobalMaxPooling2D())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(128, activation='relu', name='FEATURES'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    return model


class inceptionModule(tf.keras.layers.Layer):
    def __init__(self, n):
        super(inceptionModule,self).__init__()
        self.n = n
    def __call__(self, x_input):

        # Conv 1x1
        conv_1x1 = tf.keras.layers.Conv2D(self.n, (1, 1), padding='same', activation='relu')(x_input)

        # Conv 3x3
        conv_3x3 = tf.keras.layers.Conv2D(self.n, (1, 1), padding='same', activation='relu')(x_input)
        conv_3x3 = tf.keras.layers.Conv2D(self.n, (3, 3), padding='same', activation='relu')(conv_3x3)

        # Conv 5x5
        conv_5x5 = tf.keras.layers.Conv2D(self.n, (1, 1), padding='same', activation='relu')(x_input)
        conv_5x5 = tf.keras.layers.Conv2D(self.n, (3, 3), padding='same', activation='relu')(conv_5x5)

        # pool + proj
        pool = tf.keras.layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x_input)
        pool = tf.keras.layers.Conv2D(self.n, (1, 1), padding='same', activation='relu')(pool)

        output = tf.keras.layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pool], axis=3)

        return output



def Inception(input_shape1,
                input_shape2,
                  output_shape):


    input_shape = (input_shape1, input_shape2)
    input_layer = tf.keras.layers.Input(input_shape)

    reshape_layer = tf.keras.layers.Reshape(target_shape=(input_shape1, input_shape2, 1))(input_layer)

    x = tf.keras.layers.Conv2D(32, (6, 4), padding='same', strides=(2, 2), activation='relu')(reshape_layer)
    x = tf.keras.layers.MaxPooling2D((3, 2), padding='same', strides=(1, 1))(x)
    x = tf.keras.layers.BatchNormalization()(x)


    x = inceptionModule(32)(x)
    x = inceptionModule(64)(x)
    x = tf.keras.layers.MaxPooling2D((3, 2))(x)

    x = inceptionModule(64)(x)
    x = inceptionModule(128)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    x = tf.keras.layers.Dense(256, activation='relu')(x)

    x = tf.keras.layers.Dense(output_shape, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_layer], outputs=[x])
    return model


class residualModule(tf.keras.layers.Layer):
    def __init__(self, n_filters):
        super(residualModule,self).__init__()
        self.n_filters = n_filters
        self.merge_input = tf.keras.layers.Conv2D(
                self.n_filters, (1, 1), padding='same',
                activation='relu',
                kernel_initializer='he_normal')
        self.conv1 = tf.keras.layers.Conv2D(
            self.n_filters, (3, 3), padding='same',
            activation='relu',
            kernel_initializer='he_normal')
        self.conv2 = tf.keras.layers.Conv2D(
            self.n_filters, (3, 3), padding='same',
            activation='linear',
            kernel_initializer='he_normal')
        self.BatchNormalization = tf.keras.layers.BatchNormalization()
        self.activation = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()

    def __call__(self, layer_in):
        merge_input = layer_in
        # check if the number of filters needs to be increased
        if layer_in.shape[-1] != self.n_filters:
            merge_input = self.merge_input(layer_in)

        conv1 = self.conv1(layer_in)
        conv2 = self.conv2(conv1)

        # add filters
        layer_out = self.add([conv2, merge_input])
        layer_out = self.BatchNormalization(layer_out)
        layer_out = self.activation(layer_out)

        return layer_out



def resNet(input_shape1,
                input_shape2,
                  output_shape):

    input_shape = (input_shape1, input_shape2)

    input_layer = tf.keras.Input(shape=input_shape)

    reshape_layer = tf.keras.layers.Reshape(input_shape=input_shape, target_shape=(input_shape1, input_shape2, 1))(input_layer)

    x = tf.keras.layers.Conv2D(64, (6, 4), padding='same', strides=(2, 2), activation='relu')(reshape_layer)
    x = tf.keras.layers.MaxPool2D((3, 2), padding='same', strides=(1, 1))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = residualModule(64)(x)
    x = residualModule(128)(x)
    x = tf.keras.layers.MaxPool2D((3, 2))(x)

    x = residualModule(128)(x)
    x = residualModule(256)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    x = tf.keras.layers.Dense(512, activation='relu')(x)

    x = tf.keras.layers.Dense(output_shape, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_layer], outputs=[x])
    return model


def rnn_att_model(input_shape1,
                input_shape2,
                  output_shape,
                  cnn_features=10,
                  rnn='LSTM',
                  multi_rnn=True,
                  attention=True,
                  dropout=0.2):

    # Fetch input
    input_shape = (input_shape1, input_shape2)
    inputs = tf.keras.Input(shape=input_shape)
    # reshape = tf.keras.layers.Reshape(
    #     input_shape=input_shape, target_shape=(input_shape1, input_shape2, 1))(inputs)
    # # Normalization Layer
    # layer_out = tf.keras.layers.BatchNormalization()(reshape)
    #
    # # Convolutional Layer
    # layer_out = tf.keras.layers.Conv2D(cnn_features, kernel_size=(3, 3),
    #                                    padding='same', activation='relu')(layer_out)
    # layer_out = tf.keras.layers.BatchNormalization()(layer_out)
    # layer_out = tf.keras.layers.Conv2D(1, kernel_size=(3, 3),
    #                                    padding='same', activation='relu')(layer_out)
    # layer_out = tf.keras.layers.BatchNormalization()(layer_out)
    # layer_out = tf.keras.layers.Lambda(
    #     lambda x: tf.keras.backend.squeeze(x, -1), name='squeeze_dim')(layer_out)

    # LSTM Layer
    if rnn not in ['LSTM', 'GRU']:
        raise ValueError(
            'rnn should be equal to LSTM or GRU. No model generated...')

    if rnn == 'LSTM':
        layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            400, return_sequences=True, dropout=dropout))(inputs)
        if multi_rnn:
            layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                400, return_sequences=True, dropout=dropout))(layer_out)

    # GRU Layer
    if rnn == 'GRU':
        layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
            400, return_sequences=True, dropout=dropout))(inputs)
        if multi_rnn:
            layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                400, return_sequences=True, dropout=dropout))(layer_out)

    # Attention Layer
    if attention:
        query, value = tf.keras.layers.Lambda(
            lambda x: tf.split(x, num_or_size_splits=2, axis=2))(layer_out)
        layer_out = tf.keras.layers.Attention(name='Attention')([query, value])

    # Classification Layer
    outputs = tf.keras.layers.Flatten()(layer_out)
    outputs = tf.keras.layers.Dense(512, activation='relu')(outputs)
    outputs = tf.keras.layers.Dense(output_shape, activation='softmax')(outputs)

    # Output Model
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model


def resNet_LSTM(input_shape1,
                input_shape2,
                  output_shape,
                  cnn_features=10,
                  rnn='LSTM',
                  multi_rnn=True,
                  attention=True,
                  dropout=0.2):
    """
    Neural network with residual blocks
    Accuracy = 0.96
    """

    input_shape = (input_shape1, input_shape2)

    input_layer = tf.keras.Input(shape=input_shape)

    reshape_layer = tf.keras.layers.Reshape(input_shape=input_shape, target_shape=(input_shape1, input_shape2, 1))(input_layer)

    x = tf.keras.layers.Conv2D(64, (6, 4), padding='same', strides=(2, 2), activation='relu')(reshape_layer)
    x = tf.keras.layers.MaxPool2D((3, 2), padding='same', strides=(1, 1))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = residualModule(64)(x)
    #x = residualModule(128)(x)
    x = tf.keras.layers.MaxPool2D((3, 2))(x)

    x = residualModule(128)(x)
    #x = residualModule(256)(x)

    layer_out = tf.keras.layers.Conv2D(1, kernel_size=(3, 3),
                                       padding='same', activation='relu')(x)
    layer_out = tf.keras.layers.BatchNormalization()(layer_out)
    layer_out = tf.keras.layers.Lambda(
        lambda x: tf.keras.backend.squeeze(x, -1), name='squeeze_dim')(layer_out)

    x = tf.keras.layers.Dense(218, activation='relu')(layer_out)

    # LSTM Layer
    if rnn not in ['LSTM', 'GRU']:
        raise ValueError(
            'rnn should be equal to LSTM or GRU. No model generated...')

    if rnn == 'LSTM':
        layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            400, return_sequences=True, dropout=dropout))(x)
        if multi_rnn:
            layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                400, return_sequences=True, dropout=dropout))(layer_out)

    # GRU Layer
    if rnn == 'GRU':
        layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
            400, return_sequences=True, dropout=dropout))(x)
        if multi_rnn:
            layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                400, return_sequences=True, dropout=dropout))(layer_out)

    # Attention Layer
    if attention:
        query, value = tf.keras.layers.Lambda(
            lambda x: tf.split(x, num_or_size_splits=2, axis=2))(layer_out)
        layer_out = tf.keras.layers.Attention(name='Attention')([query, value])

    # Classification Layer
    outputs = tf.keras.layers.Flatten()(layer_out)
    outputs = tf.keras.layers.Dense(512, activation='relu')(outputs)
    outputs = tf.keras.layers.Dense(output_shape, activation='softmax')(outputs)

    model = tf.keras.Model(inputs=[input_layer], outputs=[outputs])
    return model




def LSTM_RES(input_shape1,
                input_shape2,
                  output_shape,
                  cnn_features=10,
                  rnn='LSTM',
                  multi_rnn=True,
                  attention=True,
                  dropout=0.2):
    """
    Neural network with residual blocks
    Accuracy = 0.96
    """

    input_shape = (input_shape1, input_shape2)

    input_layer = tf.keras.Input(shape=input_shape)


    # LSTM Layer
    if rnn not in ['LSTM', 'GRU']:
        raise ValueError(
            'rnn should be equal to LSTM or GRU. No model generated...')

    if rnn == 'LSTM':
        layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
            400, return_sequences=True, dropout=dropout))(input_layer)
        if multi_rnn:
            layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
                400, return_sequences=True, dropout=dropout))(layer_out)

    # GRU Layer
    if rnn == 'GRU':
        layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
            400, return_sequences=True, dropout=dropout))(input_layer)
        if multi_rnn:
            layer_out = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(
                400, return_sequences=True, dropout=dropout))(layer_out)

    # Attention Layer
    if attention:
        query, value = tf.keras.layers.Lambda(
            lambda x: tf.split(x, num_or_size_splits=2, axis=2))(layer_out)
        layer_out = tf.keras.layers.Attention(name='Attention')([query, value])


    reshape_layer = tf.keras.layers.Reshape(input_shape=[input_shape1,400], target_shape=(input_shape1, 400, 1))(layer_out)

    x = tf.keras.layers.Conv2D(64, (6, 4), padding='same', strides=(2, 2), activation='relu')(reshape_layer)
    x = tf.keras.layers.MaxPool2D((3, 2), padding='same', strides=(1, 1))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = residualModule(64)(x)
    x = residualModule(128)(x)
    x = tf.keras.layers.MaxPool2D((3, 2))(x)

    x = residualModule(128)(x)
    x = residualModule(256)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    x = tf.keras.layers.Dense(512, activation='relu')(x)

    x = tf.keras.layers.Dense(output_shape, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_layer], outputs=[x])
    return model

In [ ]:
path = '{}/test_set/sequences/*.txt'.format(main_folder)
gesture_data_test = read_shrec_2021(path)
path = '{}/training_set/sequences/*.txt'.format(main_folder)
gesture_data_train = read_shrec_2021(path)

In [ ]:
max_length = 250

fname_ = '{}/training_set/annotations_revised_training.txt'.format(main_folder)
single_gesture_train, gesture_class_train = read_annotation(fname_,gesture_data_train,max_length)
fname_ = '{}/test_set/annotations_revised.txt'.format(main_folder)
single_gesture_test, gesture_class_test = read_annotation(fname_,gesture_data_test,max_length)

In [ ]:
import time
index = 48
output_dir = '{}/processed_test'.format(main_folder)
gesture_name = ['ONE','TWO','THREE','FOUR','OK','MENU', 'LEFT','RIGHT','CIRCLE','V','CROSS','GRAB','PINCH','TAP','DENY', 'KNOB','EXPAND']
selected_classes = ['ONE','TWO','THREE','FOUR','OK','MENU', 'LEFT','RIGHT','CIRCLE','V','CROSS','GRAB','PINCH','TAP','DENY', 'KNOB','EXPAND']
# select specific classes to form the training data, this is to simulate those non selected classes as non-gestures
selected_classes_idx = [gesture_name.index(x) for x in selected_classes]
for i in range(len(gesture_class_train)):
    if gesture_class_train[i] in selected_classes_idx:
        index+=1
        output_filename = '{}/{}_{}.csv'.format(output_dir, index,gesture_class_train[i])
        # print(output_filename)
        np.savetxt(output_filename, single_gesture_train[i], fmt='%1.4f', delimiter=",")

In [ ]:
selected_joint =['palm', 'thumbA', 'thumbB', 'thumbEnd', 'indexA', 'indexB', 'indexC', 'indexEnd', 'middleA', 'middleB', 'middleC', 'middleEnd', 'ringA', 'ringB', 'ringC', 'ringEnd', 'pinkyA', 'pinkyB', 'pinkyC', 'pinkyEnd']
# Select specific joints from the training and testing data
selected_gesture_train = [select_joint(selected_joint,x,pos = True, quat= True) for x in single_gesture_train]
selected_gesture_test = [select_joint(selected_joint,x,pos = True, quat= True) for x in single_gesture_test]

In [ ]:
# select the model
Model= 'resNet'
# set model and data parameters
max_length = 250
batch_size = 64
input_dim = len(selected_joint)*7
output_dim = 17
EPOCHS = 100
learning_rate = 0.0001
drop_out = 0.3

In [ ]:
train_data = pad_data(input_dim, selected_gesture_train,max_length)
assert not np.any(np.isnan(train_data))
test_data = pad_data(input_dim,selected_gesture_test,max_length)
assert not np.any(np.isnan(test_data))
train_label, test_label = gesture_class_train, gesture_class_test

In [ ]:


train_data, train_label = multiplier(train_data, train_label, multi=1)

train_data, train_label = shuffle(train_data, train_label)


train_data = np.asarray(train_data, dtype=np.float32)
test_data = np.asarray(test_data, dtype=np.float32)

train_label = np.asarray(train_label, dtype=np.int32)
test_label = np.asarray(test_label, dtype=np.int32)

train_data, train_label = shuffle(train_data, train_label)
test_data, test_label = shuffle(test_data, test_label)

train_data = np.asarray(train_data, dtype=np.float32)
test_data = np.asarray(test_data, dtype=np.float32)

train_label = np.asarray(train_label, dtype=np.int32)
test_label = np.asarray(test_label, dtype=np.int32)


In [ ]:
# select a model and set the model parameters
model = model_selection(Model,drop_out,max_length,input_dim ,output_dim)

save_path = main_folder
if not os.path.exists(save_path):
    os.makedirs(save_path)

# log info

print('max_length {}'.format(max_length))
print('batch_size {}'.format(batch_size))
print('EPOCHS {}'.format(str(EPOCHS)))
print('learning_rate {}'.format(str(learning_rate)))
print('\n')

## Training

In [ ]:
train_classifier(
    model,
    Model,
    save_path,
    EPOCHS,
    learning_rate,
    train_data,
    train_label,
    test_data,
    test_label,
    batch_size
)

# Online Evaluation

In [ ]:
from tensorflow import keras
import editdistance



def load_model(save_path):
    SAVE_PATH_model = save_path + '/Model'
    model = keras.models.load_model(SAVE_PATH_model, custom_objects={'tf': tf})
    print('loaded model from {}'.format(save_path))
    return model

def online_rec(model,max_length, window_step, input_test):
    window_width = max_length
    outputs_list = []
    y_pred_list = []
    y_pred_value_list = []

    for window in np.arange(0, int(len(input_test) - window_width) + 1, window_step):
        window = int(window)

        input_slice = np.expand_dims(input_test[window:window + window_width], axis=0)

        Data = tf.data.Dataset.from_tensor_slices(
            (input_slice)
        )
        Data = Data.batch(1).prefetch(buffer_size=1)

        outputs = model.predict(Data, verbose=0)
        outputs_softmax = tf.nn.softmax(outputs)
        y_pred = np.argmax(outputs_softmax, axis=1)
        y_pred_value = np.max(outputs_softmax)

        outputs_list.append(outputs_softmax)
        y_pred_list.append(int(y_pred))
        y_pred_value_list.append(float(y_pred_value))

    return y_pred_list, outputs_list, y_pred_value_list


def eval_accuracy_nttd(y_pred_list, y_true, y_pred_value_list, threshold=0.6, recur_threshold=20,max = 0.21, min = 0.08):
    pred_list = []
    y_pred_value_list_ = (np.array(y_pred_value_list) - min) / (max - min)

    recur = 0
    frame_idx = []
    y_pred_ = y_pred_list[0]
    for i, value in enumerate(y_pred_value_list_):

        if value > threshold:
            recur += 1
            if recur > recur_threshold:
                y_pred = int(y_pred_list[i])
                if y_pred != y_pred_:
                    if y_pred != 0:
                        y_pred_ = y_pred
                        frame_idx.append(i)
                        frame_idx.append(i + 1)
                        pred_list.append(y_pred)
                        recur = 0

    y_pred = np.asarray(pred_list, dtype=np.int64)

    y_true = np.asarray(y_true, dtype=np.int64)

    accuracy = 1 - editdistance.eval(y_pred.tostring(), y_true.tostring()) / len(y_true.tostring())

    print('Pred {}'.format(y_pred))
    print('True {}'.format(y_true))
    print('Accuracy {}'.format(accuracy))

    return y_pred, y_true, accuracy, frame_idx

def evaluate_NTtD(classes,frame_idx,frame_sequence,y_true,y_pred):
    frame_pred = (np.array(frame_idx) + 300)[::2]
    frame_true = frame_sequence
    for j, b in enumerate(y_true):
        for i, a in enumerate(y_pred):
            start = frame_true[j * 2]
            end = frame_true[j * 2 + 1]
            #         print(frame_pred[i])

            if a == b and start < frame_pred[i] < end:
                NTtD = (frame_pred[i] - start + 1) / (end - start + 1)
                print('{} is recognized correctly at {} seconds, with NTtD {}'.format(classes[b].upper(),
                                                                                      frame_pred[i] / 72, NTtD))


def online_plot(outputs_list, input_test,input_label, frame_sequence):

    outputs_list_ = np.squeeze(outputs_list, axis=1)
    color_list_11 = ['b-', 'g-', 'r-', 'c-', 'm-', 'y-', 'k-',
                     'b--', 'g--', 'r--', 'c--', 'm--', 'y--', 'k--',
                     'b-.', 'g-.', 'r-.', 'c-.', 'm-.', 'y-.', 'k-.',
                     'b:', 'g:', 'r:', 'c:', 'm:', 'y:', 'k:', 'b.']

    list_ = np.zeros((17, len(input_test)))
    for i, index in enumerate(frame_sequence):
        if i % 2 == 0:
            list_[input_label[int(i / 2)]][frame_sequence[i]:frame_sequence[i + 1]] = 1

    list__ = np.transpose(list_)

    x_axis = np.arange(np.shape(outputs_list_)[0])
    x_axis_tile = np.tile(x_axis, (np.shape(outputs_list_)[1], 1))
    x_axis_tile_t = np.transpose(x_axis_tile)

    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=2, sharex=True)

    for i in range(17):
        if i == 0:
            continue
        ax1.plot(np.concatenate((np.arange(300) / 72, x_axis_tile_t[:, i] / 72 + 300 / 72)),
                 np.concatenate((np.ones(300) *np.mean(outputs_list_[:, 0]), outputs_list_[:, i])), color_list_11[i])

    for i in range(17):
        if i == 0:
            continue
        ax2.plot(np.concatenate((np.arange(300) / 72, x_axis_tile_t[:-1, i] / 72 + 300 / 72))[:len(list__)], list__[:, i],
                 color_list_11[i])

    plt.show()

    # fig.savefig('plot/online_{}.png'.format('_'.join([str(i) for i in input_label])))

In [ ]:
from tensorflow import keras
from sklearn.metrics import (
    confusion_matrix, accuracy_score,
    precision_score, recall_score,
    f1_score
)


def eval_classifier(Model,SAVE_PATH,test_data, test_label, batch_size):

    ## test load
    Data,_,_, y_true  = getDataTest(test_data, test_label, batch_size)
    SAVE_PATH_model = SAVE_PATH + 'Model'
    model = keras.models.load_model(SAVE_PATH_model, custom_objects={'tf': tf})
    print('Model loaded: {}'.format(Model))

    out = model.predict(Data, verbose=1)
    y_pred = np.argmax(out,axis=1)
    y_true = y_true.astype(np.int32)

    print('Real Data for validation')
    print('Accuracy: {:.4f}'.format(accuracy_score(y_pred, y_true)))
    print('Precision: {:.4f}'.format(precision_score(y_pred, y_true, average='macro')))
    print('Recall: {:.4f}'.format(recall_score(y_pred, y_true, average='macro')))
    print('F1 score: {:.4f}'.format(f1_score(y_pred, y_true, average='macro')))
    print('-------------------------------------------------------------------')
    print('\n')
    cm = confusion_matrix(y_pred, y_true)
    np.save(SAVE_PATH + '/confusion_matrix.npy', cm)

    return out,[accuracy_score(y_pred, y_true), precision_score(y_pred, y_true, average='macro'),
            recall_score(y_pred, y_true, average='macro'), f1_score(y_pred, y_true, average='macro') ]


def confusion_matrix_(cm,save_path,classes):
    cm = np.load(save_path + '/confusion_matrix.npy')


    target_names = list(classes)
    title = 'Confusion matrix'
    cmap = None
    normalize = True

    accuracy = np.trace(cm) / float(np.sum(cm))
    # np.save(SAVE_PATH+ 'confusion_matrix.npy',cm)
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(10, 10))
    # plt.imshow(cm, interpolation='nearest', cmap=cmap)
    # plt.title(title, fontsize=30)
    # plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90, fontsize=20)
        plt.yticks(tick_marks, target_names, fontsize=20)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 10 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.imshow(cm, cmap=cmap)
    # plt.grid(None)
    plt.tight_layout()
    # plt.ylabel('True label', fontsize=50)
    # plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass), fontsize=30)
    # plt.show()
#     plt.savefig('shrec21_confusion_matrix.pdf')
    # plt.close()


In [ ]:
model = load_model(save_path)

In [ ]:
# evaluating
_,_ = eval_classifier(Model,save_path, test_data, test_label,batch_size)

In [ ]:
classes = ['ONE','TWO','THREE','FOUR','OK','MENU', 'LEFT','RIGHT','CIRCLE','V','CROSS','GRAB','PINCH','TAP','DENY', 'KNOB','EXPAND']

In [ ]:
cm = np.load(save_path + '/confusion_matrix.npy')
cm1 = np.concatenate(([[0 , 0 ,0  ,0 , 0 , 0  ,0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0  ,0]],cm),0)
cm2=np.concatenate(([[16 , 0 ,0  ,0 , 0 , 0  ,0 , 0 , 0 , 0 , 0 , 0,  0 , 0 , 0 , 0  ,0,0]],cm1.T),0).T

In [ ]:
confusion_matrix_(cm2,save_path,classes)

In [ ]:
def read_annotation_idx(fname_,idx):
    gesture_name = ['ONE','TWO','THREE','FOUR','OK','MENU', 'LEFT',
                    'RIGHT','CIRCLE','V','CROSS','GRAB','PINCH','TAP','DENY', 'KNOB','EXPAND']
    with open(fname_) as f:
        lines = f.readlines()

    gesture_name_list = []
    start_idx_list = []
    end_idx_list = []
    annotation_idx_list = []
    for i in range(len(lines)):

        gesture_name_list.append(lines[i].split(';')[1:-1][::3])
        start_idx_list.append(lines[i].split(';')[2:-1][::3])
        end_idx_list.append(lines[i].split(';')[0:-1][::3][1:])
        annotation_idx_list.append(int(lines[i].split(';')[0]))

    gesture_name_list = [x for _, x in sorted(zip(annotation_idx_list, gesture_name_list))]
    start_idx_list = [x for _, x in sorted(zip(annotation_idx_list, start_idx_list))]
    end_idx_list = [x for _, x in sorted(zip(annotation_idx_list, end_idx_list))]

    label = gesture_name_list[idx]
    start_idx_list_ = start_idx_list[idx]
    end_idx_list_ = end_idx_list[idx]
    frame_sequence = [[int(start_idx_list_[i]),int(end_idx_list_[i])] for i in range(len(start_idx_list_))]

    return label, frame_sequence

In [ ]:
pos = True
quat = True

fname_ = '{}/test_set/annotations_revised.txt'.format(main_folder)

idx = 109
filename = '{}/test_set/sequences/{}.txt'.format(main_folder,idx)
data_ = genfromtxt(filename, delimiter=';')
data = np.delete(data_, -1, axis=1)
# 180;THREE;185;303;LEFT;514;534;PINCH;815;843;ONE;1047;1112;KNOB;1298;1320;
data = select_joint(selected_joint,data,pos, quat)
label, frame_sequence = read_annotation_idx(fname_,idx-109)
frame_sequence = np.reshape(frame_sequence,(-1))

y_true = [classes.index(x) for x in label]
print(frame_sequence)
print(y_true)
print(label)

In [ ]:
import tqdm
def online_rec(model,max_length, window_step, input_test):
    window_width = max_length
    outputs_list = []
    y_pred_list = []
    y_pred_value_list = []

    for window in tqdm.tqdm(np.arange(0, int(len(input_test) - window_width) + 1, window_step)):
        window = int(window)

        input_slice = np.expand_dims(input_test[window:window + window_width], axis=0)

        Data = tf.data.Dataset.from_tensor_slices(
            (input_slice)
        )
        Data = Data.batch(1).prefetch(buffer_size=1)

        outputs = model.predict(Data, verbose=0)
        outputs_softmax = tf.nn.softmax(outputs)
        y_pred = np.argmax(outputs_softmax, axis=1)
        y_pred_value = np.max(outputs_softmax)

        outputs_list.append(outputs_softmax)
        y_pred_list.append(int(y_pred))
        y_pred_value_list.append(float(y_pred_value))

    return y_pred_list, outputs_list, y_pred_value_list

In [ ]:
window_step = 1
y_pred_list, outputs_list, y_pred_value_list = online_rec(model,max_length, window_step, data)

In [ ]:
y_pred, y_true, accuracy, frame_idx= eval_accuracy_nttd(y_pred_list, y_true, y_pred_value_list, threshold=0.5, recur_threshold=10, max = 0.14, min = 0.06)

In [ ]:
evaluate_NTtD(classes,frame_idx,frame_sequence,y_true,y_pred)

In [ ]:
online_plot(outputs_list, data,y_true, frame_sequence)